##  주가 데이터 가져와서 db에 넣기

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


In [25]:
driver = webdriver.Chrome()
url = 'https://finance.naver.com/item/sise_time.naver?code=005930&thistime=20220209103212&page=2'  # 
driver.get(url)
time.sleep(0.5)
html = driver.page_source

df = pd.read_html(html)[0]  # 리스트에 담겨있어서 빼줘야 함
df.dropna(inplace= True)
df.head()

# driver.close()

,체결시각,체결가,전일비,매도,매수,거래량,변동량
1,10:22,74500.0,1000.0,74500.0,74400.0,4184451.0,3535.0
2,10:21,74400.0,900.0,74500.0,74400.0,4180916.0,5251.0
3,10:20,74500.0,1000.0,74500.0,74400.0,4175665.0,4205.0
4,10:19,74500.0,1000.0,74500.0,74400.0,4171460.0,4224.0
5,10:18,74500.0,1000.0,74500.0,74400.0,4167236.0,97070.0


In [26]:
df_code = pd.read_csv('./data/universe_code.csv')
codes = df_code.종목코드.apply(lambda x: x[1:]).values
# codes

In [27]:
import MySQLdb

db = MySQLdb.connect(host = 'localhost', port=3306, user='root', password='1234')
cursor = db.cursor()

In [28]:
sql = "show databases;"
pd.read_sql(sql,db)

,Database
0,information_schema
1,budongsan
2,employees
3,fin_db
4,gmarketdb
5,mysql
6,performance_schema
7,sakila
8,shopdb
9,sqldb


In [29]:
sql = " use fin_db;"
cursor.execute(sql)

0

In [30]:
# 데이터베이스에 stock_table 이라는 테이블 생성
sql= """
create table if not exists stock_table(
code char(6),
date date,
close float,
margin varchar(50),
start float,
high float,
low float,
volume float
);"""

cursor.execute(sql)

0

In [31]:
url = 'https://finance.naver.com/item/sise_day.naver?code={}&page={}'.format(code, num)

In [37]:
url = 'https://finance.naver.com/item/sise_day.naver?code={}&page={}'
driver = webdriver.Chrome()

for code in codes[:3]:
    print(code, '================')
    for i in range(1,10):  # 10페이지 100일치
        driver.get(url.format(code,i))
        time.sleep(0.5)
        html = driver.page_source
        
        df = pd.read_html(html)[0]
        df.dropna(inplace=True)
        print(df)
        print("-----------------")
        
        for i in range(len(df)):
            code = code
            date = df.iloc[i][0]
            close = df.iloc[i][1]
            margin = df.iloc[i][2]
            start = df.iloc[i][3]
            high = df.iloc[i][4]
            low = df.iloc[i][5]
            volume = df.iloc[i][6]
            
            sql="""insert into stock_table values('{},{},{},{},{},{},{},{}');""",format(code,date,close,margin,start,high,low,volume)
            
            
        

000210 ================
            날짜       종가     전일비       시가       고가       저가       거래량
1   2022.02.09  60200.0   600.0  59700.0  60400.0  59000.0   12276.0
2   2022.02.08  59600.0  1000.0  59100.0  61000.0  58600.0   83130.0
3   2022.02.07  58600.0  1100.0  57500.0  58800.0  57000.0   53386.0
4   2022.02.04  57500.0   900.0  56900.0  57800.0  56600.0   34228.0
5   2022.02.03  56600.0   700.0  56300.0  57500.0  56300.0   44961.0
9   2022.01.28  55900.0  1700.0  54400.0  57500.0  53900.0   95644.0
10  2022.01.27  54200.0  2800.0  57000.0  57100.0  53900.0   97345.0
11  2022.01.26  57000.0   100.0  56600.0  57800.0  56600.0   39639.0
12  2022.01.25  57100.0  1600.0  58200.0  58500.0  56600.0  106342.0
13  2022.01.24  58700.0  1700.0  59300.0  61000.0  58300.0  120883.0
-----------------


TypeError: format expected at most 2 arguments, got 8

In [ ]:
sql = 'select * from stock_table;'

st_price = pd.read_sql(sql, db)
st_price